In [3]:
import pandas as pd
import sqlite3

In [4]:
conn = sqlite3.connect('bios-player-stats.db')

df = pd.read_excel('../processed-data/bios.xlsx', sheet_name='bios')
df.to_sql('bios', conn, index=False,if_exists="replace")

df_players = pd.read_excel('../processed-data/stacked-player-stats.xlsx', sheet_name='player-stats')
df_players.to_sql('players', conn, index=False,if_exists="replace")

curr = conn.cursor()

C:\Users\Max Brenner\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [5]:
df.head()

,Player,Age,Height,Weight,College,Country,Draft Year,Draft Round,Draft Number
0,Aaron Gordon,24,80,235,Arizona,USA,2014.0,1.0,4.0
1,Aaron Holiday,23,72,185,UCLA,USA,2018.0,1.0,23.0
2,Abdel Nader,26,77,225,Iowa State,Egypt,2016.0,2.0,58.0
3,Adam Mokoka,21,77,190,NaN,France,NaN,NaN,NaN
4,Admiral Schofield,23,77,241,Tennessee,United Kingdo ...,2019.0,2.0,42.0


In [6]:
def query_db(query):
    execute = curr.execute(query)
    cols = [column[0] for column in execute.description]
    results = pd.DataFrame.from_records(data=execute.fetchall(), columns=cols)
    return results

In [16]:
# Colleges Count
query = \
'''
SELECT College, COUNT(College) as 'How Many Attended', 
COUNT(case when "Draft Round" is 1 then 1 else null end) as 'First Draft Round',
COUNT(case when "Draft Round" is 2 then 1 else null end) as 'Second Draft Round'
FROM bios
WHERE College IS NOT NULL
GROUP BY College
HAVING COUNT(College) >= 5
ORDER BY COUNT('Number Attended') DESC, COUNT('First Round Draft') DESC, COUNT('Second Round Draft') DESC
LIMIT 10
'''
query_db(query)

,College,How Many Attended,First Draft Round,Second Draft Round
0,Kentucky,26,22,2
1,Duke,23,19,1
2,North Carolina,13,12,1
3,Kansas,12,7,4
4,UCLA,11,9,2
5,Arizona,10,7,1
6,Michigan,10,6,2
7,Indiana,9,6,1
8,Texas,9,8,1
9,Gonzaga,8,5,1


In [8]:
# College Draft Number Distribution
query = \
'''
SELECT College, AVG("Draft Number") as "Avg Draft Number", 
MIN("Draft Number") as "Best Draft Number", MAX("Draft Number") as "Worst Draft Number"
FROM bios
GROUP BY College
HAVING COUNT(College) >= 5
ORDER BY "Avg Draft Number", "Best Draft Number"
limit 5
'''
query_db(query)

,College,Avg Draft Number,Best Draft Number,Worst Draft Number
0,Duke,10.900000,1.0,31.0
1,Connecticut,12.400000,8.0,24.0
2,Wake Forest,13.200000,4.0,19.0
3,Kentucky,13.916667,1.0,45.0
4,Indiana,14.428571,2.0,42.0


In [9]:
# College Stats
query = \
'''
SELECT College, AVG(Vorp) as "Avg Vorp", AVG(WS), AVG(WS/48), AVG(PER), AVG(BPM), AVG(Obpm), AVG(Dbpm)
FROM bios
JOIN players ON players.player is bios.player
WHERE Season is "19-20"
GROUP BY College
HAVING COUNT(College) >= 5
ORDER BY "Avg Vorp" DESC
LIMIT 5
'''
query_db(query)

,College,Avg Vorp,AVG(WS),AVG(WS/48),AVG(PER),AVG(BPM),AVG(Obpm),AVG(Dbpm)
0,Connecticut,1.040000,3.020000,0.062917,17.200000,0.720000,0.300000,4.000000e-01
1,Louisville,0.900000,3.220000,0.067083,17.380000,0.540000,0.520000,4.000000e-02
2,UCLA,0.890000,2.360000,0.049167,15.150000,0.250000,-0.020000,2.700000e-01
3,Texas,0.833333,3.144444,0.065509,14.700000,0.244444,-0.488889,7.444444e-01
4,Kentucky,0.804348,2.526087,0.052627,14.791304,-1.126087,-1.121739,-3.861645e-17
